In [ ]:
import os
import pickle
import numpy as np
from glob import glob

In [ ]:
# get all results
root = '../../results/'
exp_path = '*'
dataset = '*'
architecture = '*'

mean_crs = sorted(glob(os.path.join(root, exp_path, dataset, architecture + '*_report.pickle')))
print('Found: %s reports' % len(mean_crs))

In [ ]:
def extract_values(mean_crs, keys, print_path=False):
    rd = {}
    for mcr in mean_crs:
        with open(mcr, 'rb') as f:
            d = pickle.load(f)
        if print_path:
            print('Path: %s' % mcr)
        setup = mcr.split('/')[-3:]
        exp, dataset, model = setup
        model = model.split('_run')[0]
        #dict setup
        if not exp in rd.keys():
            rd[exp] = {}
        if not dataset in rd[exp].keys():
            rd[exp][dataset] = {}
        if not model in rd[exp][dataset].keys():
            rd[exp][dataset][model] = []
        # values insert
        tmp = {}
        for k in keys:
            if k in d.keys():
                tmp[k] = d[k]
            elif '/' in k:
                k1, k2 = k.split('/')
                tmp[k] = d[k1][k2]
        rd[exp][dataset][model].append(tmp)
    return rd
    
def print_values(rd, best_key=None):
    for e in sorted(rd.keys()):
        for d in sorted(rd[e].keys()):
            for m in sorted(rd[e][d].keys()):
                print('Exp: %s | Dataset: %s | Model: %s' %(e, d, m))
                cd = rd[e][d][m]
                if not best_key is None:
                    best_i, best_val = 0, 0
                    for i in range(len(cd)):
                        curr_val = float(str(cd[i][best_key]).split(' ')[0])
                        if curr_val > best_val:
                            best_i, best_val = i, curr_val
                indices = np.arange(len(cd)) if best_key is None else [best_i]
                for i in indices:
                    for k in cd[i].keys():
                        print('Idx: %s Key: %s | Value: %s' % (i, k, cd[i][k]))
                print('=' * 80)

In [ ]:
keys = ['weighted avg/f1-score']
rd = extract_values(mean_crs, keys)
print_values(rd, best_key=keys[0])